```{contents}
```
## **Autonomous Agent in LangGraph**

An **Autonomous Agent** in LangGraph is a **self-directed, persistent, decision-making system** that can perceive its environment, reason about goals, take actions using tools, observe results, and iteratively improve its behavior — **without continuous human intervention**.

Formally, it is implemented as a **cyclic, stateful execution graph** that embodies the control loop:

> **Perceive → Reason → Act → Observe → Learn → Repeat**

---

### **1. Why Autonomous Agents Need LangGraph**

Traditional LLM chains are **linear and stateless**.
Autonomous agents require:

| Capability           | Why Needed                         |
| -------------------- | ---------------------------------- |
| Persistent state     | Remember goals, history, progress  |
| Cycles               | Continuous reasoning & improvement |
| Dynamic control flow | React to runtime events            |
| Tool execution       | Affect external environment        |
| Error recovery       | Robust long-running operation      |
| Human override       | Safety and compliance              |

LangGraph provides these capabilities natively.

---

### **2. Core Components of an Autonomous Agent**

| Component             | Role                           |
| --------------------- | ------------------------------ |
| **State**             | Memory of the agent            |
| **Nodes**             | Cognitive and action functions |
| **Edges**             | Control flow                   |
| **Router**            | Decision-making logic          |
| **Tools**             | External actions               |
| **Memory Store**      | Long-term knowledge            |
| **Termination Logic** | Stop conditions                |

---

### **3. Canonical Agent Control Loop**

```
┌──────────┐
│ Observe  │  ←─ External environment
└────┬─────┘
     ↓
┌──────────┐
│  Reason  │  ←─ Planning / thinking
└────┬─────┘
     ↓
┌──────────┐
│   Act    │  ←─ Tool execution
└────┬─────┘
     ↓
┌──────────┐
│ Evaluate │  ←─ Success / failure
└────┬─────┘
     ↺ (loop until goal achieved)
```

---

### **4. Agent State Design**

```python
class AgentState(TypedDict):
    goal: str
    plan: str
    observation: str
    action: str
    memory: list[str]
    success: bool
```

The **state is the agent’s mind**.

---

### **5. Node Implementations**

#### Reason Node

```python
def reason(state):
    plan = llm(f"Goal: {state['goal']}\nObservation: {state['observation']}\nPlan:")
    return {"plan": plan}
```

#### Act Node

```python
def act(state):
    result = tool_executor(state["plan"])
    return {"action": result}
```

#### Observe Node

```python
def observe(state):
    obs = environment_feedback(state["action"])
    return {"observation": obs}
```

#### Evaluate Node

```python
def evaluate(state):
    success = "done" in state["observation"]
    return {"success": success}
```

---

### **6. Constructing the Autonomous Graph**

```python
from langgraph.graph import StateGraph, END

builder = StateGraph(AgentState)

builder.add_node("reason", reason)
builder.add_node("act", act)
builder.add_node("observe", observe)
builder.add_node("evaluate", evaluate)

builder.set_entry_point("reason")

builder.add_edge("reason", "act")
builder.add_edge("act", "observe")
builder.add_edge("observe", "evaluate")

def route(state):
    return END if state["success"] else "reason"

builder.add_conditional_edges("evaluate", route, {
    "reason": "reason",
    END: END
})

agent = builder.compile()
```

---

### **7. Running the Autonomous Agent**

```python
result = agent.invoke({
    "goal": "Find the latest stock price of Apple and summarize",
    "plan": "",
    "observation": "",
    "action": "",
    "memory": [],
    "success": False
})
```

---

### **8. Agent Variants in LangGraph**

| Variant            | Description                   |
| ------------------ | ----------------------------- |
| Reactive Agent     | Simple sense–act              |
| Planner–Executor   | Separate planning & acting    |
| Reflective Agent   | Self-critique & correction    |
| Multi-Agent System | Multiple collaborating agents |
| Hierarchical Agent | Supervisor + workers          |
| Self-Healing Agent | Error detection & recovery    |

---

### **9. Safety & Production Controls**

| Control          | Purpose                |
| ---------------- | ---------------------- |
| Max iterations   | Prevent infinite loops |
| Timeouts         | Kill hung agents       |
| Checkpointing    | Crash recovery         |
| Human interrupts | Manual override        |
| Audit logs       | Compliance             |

```python
agent.invoke(input, config={"recursion_limit": 20})
```

---

### **10. Why LangGraph is Ideal for Autonomous Agents**

| Feature          | Importance         |
| ---------------- | ------------------ |
| Persistent state | Memory             |
| Cycles           | Autonomy           |
| Dynamic routing  | Adaptability       |
| Tool integration | Real-world action  |
| Observability    | Debugging & safety |

---

### **11. Conceptual Summary**

An autonomous agent in LangGraph is not a prompt —
it is a **living control system** implemented as a **stateful cyclic graph**.


### Demonstration

In [1]:
from typing import TypedDict
from langgraph.graph import StateGraph, END

# -------------------------
# 1. Define Agent State
# -------------------------
class AgentState(TypedDict):
    goal: str
    plan: str
    observation: str
    action: str
    success: bool

# -------------------------
# 2. Define Agent Nodes
# -------------------------
def reason(state: AgentState):
    plan = f"Search information about: {state['goal']}"
    return {"plan": plan}

def act(state: AgentState):
    # Simulated tool execution
    action_result = f"Found info related to '{state['goal']}'"
    return {"action": action_result}

def observe(state: AgentState):
    observation = f"Observation: {state['action']}"
    return {"observation": observation}

def evaluate(state: AgentState):
    success = "Found" in state["observation"]
    return {"success": success}

# -------------------------
# 3. Build Cyclic Graph
# -------------------------
builder = StateGraph(AgentState)

builder.add_node("reason", reason)
builder.add_node("act", act)
builder.add_node("observe", observe)
builder.add_node("evaluate", evaluate)

builder.set_entry_point("reason")

builder.add_edge("reason", "act")
builder.add_edge("act", "observe")
builder.add_edge("observe", "evaluate")

def route(state: AgentState):
    return END if state["success"] else "reason"

builder.add_conditional_edges("evaluate", route, {"reason": "reason", END: END})

agent = builder.compile()

# -------------------------
# 4. Run the Autonomous Agent
# -------------------------
result = agent.invoke({
    "goal": "Explain LangGraph autonomous agents",
    "plan": "",
    "observation": "",
    "action": "",
    "success": False
})

print(result)


{'goal': 'Explain LangGraph autonomous agents', 'plan': 'Search information about: Explain LangGraph autonomous agents', 'observation': "Observation: Found info related to 'Explain LangGraph autonomous agents'", 'action': "Found info related to 'Explain LangGraph autonomous agents'", 'success': True}
